# TenSEAL CKKS - Introduction

Homomorphic Encryption  is an encryption technique that allows computations to be made on ciphertexts and generates results that when decrypted, correspond to the results of the same computations made on plaintexts. 

**[CKKS - Cheon-Kim-Kim-Song](https://www.inferati.com/blog/fhe-schemes-ckks)** which is Homomorphic Encryption for Arithmetic of Approximate Numbers (HEAAN), was proposed to offer homomorphic computation on real numbers. The main idea is to consider the noise, a.k.a. error $ e$ , which is introduced in Ring-Learning with Errors (Ring-LWE) based FHE schemes for security purposes, as part of the message $\mu $ (which we call here payload) we want to encrypt. The payload and the noise are combined to generate the plaintext ($\mu + e$) that we encrypt.

Next we will look at the most important object of the library, the TenSEALContext for CKKS HE Scheme :

In [1]:
import tenseal as ts
import numpy as np
import time as time
%load_ext memory_profiler

In [2]:
# Setup TenSEAL context
context = ts.context(
            ts.SCHEME_TYPE.CKKS,
            poly_modulus_degree=8192,
            coeff_mod_bit_sizes=[60, 40, 40, 60]
          )
"""
Genrate Galois Keys : Evaluation keys for the homomorphic rotation operation,
which is the cyclic shift operations for rows of the encrypted matrix in one
ciphertext of the BFV scheme and for encrypted message vector in that of the 
CKKS scheme.
"""
context.generate_galois_keys()
context.global_scale = 2**40

In [3]:
# Add two vectors
v1 = [5, 1, 2, 3, 4]
v2 = [4, 3, 2, 1, 5]

In [4]:
# Encrypt the vectors using the CKKS scheme
enc_v1 = ts.ckks_vector(context, v1)
enc_v2 = ts.ckks_vector(context, v2)

## Addition

In [5]:
add_result = enc_v1 + enc_v2
add_result.decrypt() # ~ [9,4,4,4,9]

[9.00000000168854,
 3.999999998110309,
 3.999999999964051,
 4.000000000037462,
 9.000000001457845]

### Time taken to perform addition

In [6]:
print("Homomorphic Addition takes:")
he_add =  %timeit -o (enc_v1 + enc_v2)
he_add_mem =  %memit -o (enc_v1 + enc_v2)

print("Vector Addition takes:")
v_add = %timeit -o (np.array(v1) + np.array(v2))
v_add_mem = %memit -o (np.array(v1) + np.array(v2))

res = he_add.best / v_add.best

print("Vector Addition is {} times faster than Homomorphic Addition".format(res))

Homomorphic Addition takes:
56.4 µs ± 358 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
peak memory: 174.73 MiB, increment: 0.11 MiB
Vector Addition takes:
1.56 µs ± 6.23 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
peak memory: 174.94 MiB, increment: 0.00 MiB
Vector Addition is 36.068307582849556 times faster than Homomorphic Addition


## Subtraction

In [7]:
subp_result = np.array(v1) - np.array(v2)
subp_result

array([ 1, -2,  0,  2, -1])

In [8]:
sub_result = enc_v1 - enc_v2
sub_result.decrypt()

[0.9999999930663853,
 -2.0000000009764403,
 2.2837109425744018e-09,
 2.000000000272445,
 -0.9999999986212919]

### Time taken to perform Subtraction

In [9]:
print("Homomorphic Subtraction takes:")
he_sub =  %timeit -o (enc_v2 - enc_v1)
he_sub_mem =  %memit -o (enc_v2 - enc_v1)


print("Vector Subtraction takes:")
v_sub = %timeit -o (np.array(v2) - np.array(v1))
v_sub_mem = %memit -o (np.array(v2) - np.array(v1))

res = he_sub.best / v_sub.best
print("Vector Subtraction is {} times faster than Homomorphic Subtraction".format(res))

Homomorphic Subtraction takes:
52.4 µs ± 228 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
peak memory: 175.69 MiB, increment: 0.00 MiB
Vector Subtraction takes:
1.61 µs ± 11.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
peak memory: 175.86 MiB, increment: 0.00 MiB
Vector Subtraction is 32.6571262505289 times faster than Homomorphic Subtraction


## Negation

In [10]:
neg_result = -enc_v1
neg_result.decrypt()

[-4.999999997377462,
 -0.9999999985669354,
 -2.0000000011238805,
 -3.0000000001549534,
 -4.000000001418278]

### Time taken to perform Negation

In [11]:
print("Homomorphic Negation takes:")
he_neg =  %timeit -o (-enc_v1)
he_neg_mem =  %memit -o (-enc_v1)

print("Vector Subtraction takes:")
v_neg = %timeit -o (-np.array(v1))
v_neg_mem = %memit -o (-np.array(v1))

res = he_neg.best / v_neg.best
print("Vector Negation is {} times faster than Homomorphic Negation".format(res))

Homomorphic Negation takes:
39.5 µs ± 214 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
peak memory: 176.54 MiB, increment: 0.00 MiB
Vector Subtraction takes:
934 ns ± 5.37 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
peak memory: 176.54 MiB, increment: 0.00 MiB
Vector Negation is 42.27770425793887 times faster than Homomorphic Negation


## Power

In [12]:
power_res = enc_v1 ** 4
power_res._decrypt()

[625.000585293541,
 1.0000009341151213,
 16.00001505487296,
 81.00007606110789,
 256.00024063446153]

### Time taken to calculate exponents and powers

In [13]:
print("Homomorphic Negation takes:")
he_pow =  %timeit -o (enc_v1 ** 4)
he_pow_mem =  %memit -o (enc_v1 ** 4)

print("Vector Subtraction takes:")
v_pow = %timeit -o (np.array(v1) ** 4)
v_pow_mem = %memit -o (np.array(v1) ** 4)

res = he_pow.best / v_pow.best
print("Vector Negation is {} times faster than Homomorphic Negation".format(res))

Homomorphic Negation takes:
6.07 ms ± 58.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
peak memory: 178.18 MiB, increment: 0.00 MiB
Vector Subtraction takes:
1.76 µs ± 7.83 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
peak memory: 178.18 MiB, increment: 0.00 MiB
Vector Negation is 3435.589111884566 times faster than Homomorphic Negation


## Dot Product

In [14]:
result = enc_v1.dot(enc_v2)
result.decrypt() # ~ [50]

[50.00000864874787]

### Time taken to perform Dot Product

In [15]:
print("Homomorphic Dot Product takes:")
he_dp =  %timeit -o (enc_v1.dot(enc_v2))
he_dp_mem =  %memit -o (enc_v1.dot(enc_v2))

print("Vector Dot Product takes:")
v_dp = %timeit -o (np.array(v1).dot(np.array(v2)))
v_dp_mem = %memit -o (np.array(v1).dot(np.array(v2)))

res = he_dp.best / v_dp.best

print("Vector Dot Product is {} times faster than Homomorphic Dot Product".format(res))

Homomorphic Dot Product takes:
9.28 ms ± 76 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
peak memory: 180.00 MiB, increment: 0.00 MiB
Vector Dot Product takes:
1.9 µs ± 18.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
peak memory: 180.00 MiB, increment: 0.00 MiB
Vector Dot Product is 4863.666414601436 times faster than Homomorphic Dot Product


## Matrix Multiplication

In [16]:
matrix = [
  [79, 0.15, 18],
  [21, -5, 64],
  [-18, -98, -3],
  [1, 9, 87],
  [88, 65 , 1],
]

In [17]:
result = enc_v1.matmul(matrix)
result.decrypt()

[735.0000984405782, 86.75001162554906, 413.0000555187746]

### Time taken to perform Matrix Multiplication

In [18]:
print("Homomorphic Matrix Multiplication takes:")
he_mm =  %timeit -o (enc_v1.matmul(matrix))
he_mm_mem =  %memit -o (enc_v1.matmul(matrix))

print("Vector Matrix Multiplication takes:")
v_mm = %timeit -o (np.matmul(v1,matrix))
v_mm_mem = %memit -o (np.matmul(v1,matrix))

res = he_mm.best / v_mm.best

print("Vector Matrix Multiplication is {} times faster than HomomorphicMatrix Multiplication".format(res))

Homomorphic Matrix Multiplication takes:
20.7 ms ± 161 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
peak memory: 195.84 MiB, increment: 0.92 MiB
Vector Matrix Multiplication takes:
5.72 µs ± 107 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
peak memory: 195.91 MiB, increment: 0.00 MiB
Vector Matrix Multiplication is 3644.0243077417385 times faster than HomomorphicMatrix Multiplication


## Conclusion

In this notebook, we saw various capabilities and operations that are supported by CKKS scheme. These include - addition, subtraction, negation, power, multiplication, dot product, polynomial evaluation and matrix multiplication. In the future work, we can start looking into another capability of the CKKSvector that allows image block to columns operation and can be used for CNN for classification. 